# Курсовая работа за 4 семестр
## Островерхов Артем, КН-202
___
# House Prices - Advanced Regression Techniques
### Формулировка задачи:
Сравнение использования различных архитектур нейронных сетей для решения задач регрессии, на примере реальной задачи с Kaggle\
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

**Цель данного проекта** — построить модель машинного обучения для прогнозирования конечной цены продажи жилых домов в городе Эймс, штат Айова. Предоставленный набор данных включает 79 признаков, описывающих различные аспекты недвижимости: от площади и количества комнат до качества отделки и расположения. Эта задача относится к типу регрессии, где необходимо предсказать непрерывную числовую переменную — цену продажи дома.
___
# Описание задачи
Для начала посмотрим на данную мне задачу и разберем, что такое регрессия и что надо знать, прежде чем начать работу.

**Регрессия** — это тип задачи машинного обучения, в которой модель учится предсказывать числовое значение на основе входных данных.

___
**Формальное определение**
В задаче регрессии требуется найти такую функцию:
$$
f : \mathbb{R}^n \to \mathbb{R}
$$
которая по входному вектору признаков:
$$
\mathbf{x} = (x_1, x_2, \dots, x_n)
$$
предсказывает выходное числовое значение:
$$
\hat{y} = f(\mathbf{x})
$$
таким образом, чтобы приближать истинное значение:
$$
y \in \mathbb{R}
$$
как можно точнее.
___
То есть можно переформулировать **цель обучения**
Найти такую функцию $f(\mathbf{x})$, которая... минимизирует ошибку предсказания.
Например, для линейной регрессии мы ищем:
$$
\hat{y} = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_n x_n = \mathbf{w}^\top \mathbf{x} + b
$$
Модель обучается путём минимизации функции потерь — например, среднеквадратичной ошибки (MSE):
$$
\mathrm{MSE} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2
$$
где:
- $m$ — количество примеров в обучающей выборке
- $\hat{y}_i$ — предсказание модели
- $y_i$ — истинное значение
___
**Особенности набора данных**

Набор данных содержит 1460 объектов в обучающей выборке и 1459 объектов в тестовой. Каждый дом описан 79 признаками, среди которых есть как числовые (GrLivArea, LotArea, YearBuilt), так и категориальные (Neighborhood, HouseStyle, Exterior1st и др.).

Перед началом обучения важно:\
	•	обработать пропущенные значения;\
	•	закодировать категориальные признаки (например, через one-hot или label encoding);\
	•	масштабировать числовые признаки (например, с помощью StandardScaler);\
	•	преобразовать целевую переменную логарифмически (для уменьшения влияния выбросов):\
$$
y = \log(\text{SalePrice})
$$
___
**Метрика оценки**

На Kaggle используется метрика RMSLE (Root Mean Squared Logarithmic Error), которая вычисляется по формуле:

$$
\text{RMSLE} = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} \left( \log(\hat{y}_i + 1) - \log(y_i + 1) \right)^2 }
$$

Данная метрика особенно полезна, когда важно уменьшить влияние больших ошибок при переоценке стоимости.
Она делает больший акцент на относительные отклонения, а не на абсолютные, и более устойчива к выбросам.
___
# Ход работы

**Цель данного этапа** — реализовать и сравнить несколько архитектур моделей регрессии, включая как классические, так и нейросетевые подходы.

В рамках эксперимента были реализованы и обучены следующие модели:

- **ElasticNet** — базовая линейная модель с регуляризацией
- **MLP (Multi-Layer Perceptron)** — полносвязная нейронная сеть
- **CNN1D (одномерная сверточная сеть)** — экспериментальное применение сверточной архитектуры к табличным данным

Для каждой архитектуры проводились:

1. Предобработка данных (кодирование, масштабирование, логарифмирование целевой переменной)
2. Обучение модели с фиксированными параметрами
3. Валидация качества предсказания по метрике RMSE
4. Визуальный анализ ошибок

Ниже приведено поэтапное описание подготовки данных и реализации каждой модели.
___
## 1. ElasticNet

Модель ElasticNet применяется в задачах линейной регрессии, когда наблюдаются следующие проблемы:

- Мультиколлинеарность признаков (сильная корреляция между переменными),
- Наличие большого количества нерелевантных или шумовых признаков,
- Необходимость интерпретируемости и устойчивости модели.

ElasticNet представляет собой обобщение двух видов регуляризации:

- L1-регуляризация (Lasso), зануляющая коэффициенты и производящая отбор признаков,
- L2-регуляризация (Ridge), снижающая чувствительность модели к коррелированным признакам.

### Математическая формулировка

Целевая функция ElasticNet объединяет обе регуляризации и имеет следующий вид:

$$
\min_{\mathbf{w}, b} \left\{ \frac{1}{2m} \sum_{i=1}^{m} \left( y_i - \mathbf{w}^\top \mathbf{x}_i - b \right)^2 + \alpha \left( \rho \|\mathbf{w}\|_1 + \frac{1 - \rho}{2} \|\mathbf{w}\|_2^2 \right) \right\}
$$

где:
- $m$ — количество объектов обучающей выборки
- $\mathbf{x}_i \in \mathbb{R}^n$ — вектор признаков для $i$-го объекта
- $y_i \in \mathbb{R}$ — соответствующее целевое значение
- $\mathbf{w} \in \mathbb{R}^n$, $b \in \mathbb{R}$ — параметры модели
- $\alpha > 0$ — коэффициент регуляризации
- $\rho \in [0, 1]$ — параметр смешивания регуляризаторов
- $\|\mathbf{w}\|_1$ — сумма абсолютных значений коэффициентов (L1-норма)
- $\|\mathbf{w}\|_2^2$ — сумма квадратов коэффициентов (L2-норма)

При $ \rho = 1 $ модель эквивалентна Lasso, при $ \rho = 0 $ — Ridge.
Значения $ \rho \in (0, 1) $ обеспечивают компромисс между разреженностью и устойчивостью.

### Подготовка данных

Перед обучением модели были выполнены следующие шаги:

- Логарифмическое преобразование целевой переменной:

  $$
  y = \log(\text{SalePrice} + 1)
  $$

- Объединение обучающей и тестовой выборок для единой предобработки,
- Логарифмирование числовых признаков с высокой скошенностью:

  $$
  x' = \log(x + 1), \quad \text{если} \quad |\text{skew}(x)| > 0.75
  $$

- Заполнение пропущенных значений:
  - медианой — для числовых признаков,
  - наиболее частой категорией — для категориальных.

- Кодирование категориальных признаков методом One-Hot Encoding,
- Масштабирование всех числовых признаков с помощью стандартизации (StandardScaler).

### Гиперпараметры и обучение

Модель обучалась с помощью метода координатного спуска с ограничением по числу итераций
($ \text{max\_iter} = 5000 $) и критерием сходимости ($ \text{tol} = 10^{-3} $).

Подбор гиперпараметров ($ \alpha $) и ($ \rho $) производился с использованием GridSearchCV по сетке:

- $ \alpha \in \{10^{-3}, 10^{-2}, 10^{-1}, 1, 10, 10^2, 10^3\} $
- $ \rho \in \{0.1, 0.5, 0.9\} $

Оптимизация осуществлялась по метрике RMSLE (Root Mean Squared Logarithmic Error):

$$
\text{RMSLE} = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} \left( \log(\hat{y}_i + 1) - \log(y_i + 1) \right)^2 }
$$

Такое логарифмическое преобразование целевой переменной снижает влияние выбросов и делает метрику более устойчивой к большим значениям.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import warnings

warnings.filterwarnings('ignore')

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

train   = pd.read_csv('train.csv', na_values=[''], keep_default_na=True)
test    = pd.read_csv('test.csv',  na_values=[''], keep_default_na=True)
test_ID = test['Id']

y = np.log1p(train['SalePrice'])
train.drop(columns='SalePrice', inplace=True)
all_data = pd.concat([train, test], ignore_index=True)

num_feats = all_data.select_dtypes(include=['int64','float64']).columns.drop('Id')
skews     = all_data[num_feats].apply(lambda col: skew(col.dropna()))
skewed    = skews[abs(skews) > 0.75].index
all_data[skewed] = np.log1p(all_data[skewed])

cat_feats = all_data.select_dtypes(include=['object']).columns.tolist()

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer,   num_feats),
    ('cat', categorical_transformer, cat_feats)
])

pipeline = Pipeline([
    ('preproc', preprocessor),
    ('reg',     ElasticNet(max_iter=5000, tol=1e-3))
])

param_grid = {
    'reg__alpha':    np.logspace(-3, 3, 7),
    'reg__l1_ratio': [0.1, 0.5, 0.9]
}

search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=rmsle_scorer,
    n_jobs=-1,
    verbose=1
)

X_train = all_data.iloc[:len(y)].drop(columns=['Id'])
search.fit(X_train, y)
print("Best parameters:", search.best_params_)

X_test    = all_data.iloc[len(y):].drop(columns=['Id'])
preds_log = search.predict(X_test)
preds     = np.expm1(preds_log)

submission            = pd.read_csv('sample_submission.csv')
submission['SalePrice'] = preds
submission.to_csv('submission.csv', index=False)
print("Saved submission.csv")

### Подробности реализации модели ElasticNet на основе кода

В данной реализации используется `ElasticNet` как базовая линейная модель с регуляризацией. Основное внимание в коде уделяется правильной настройке метрики качества, поиску гиперпараметров, а также точной обработке признаков.

---

### Настройка метрики RMSLE

В качестве основной метрики используется **RMSLE** (корень из средней квадратичной логарифмической ошибки). Она задаётся вручную:

```python
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)
```

Это кастомная метрика, основанная на mean_squared_error, но предполагающая, что логарифмирование целевой переменной уже произведено заранее. Поэтому log1p() применяется сразу к y перед обучением, а expm1() — после предсказания.

Аргумент greater_is_better=False указывает GridSearchCV, что меньшие значения метрики — лучше.

---

### Выбор гиперпараметров

Подбор гиперпараметров модели производится по двум ключевым параметрам:

- `alpha` — коэффициент регуляризации: чем он больше, тем сильнее модель штрафует большие веса. Значения подбираются логарифмически:

```python
'reg__alpha': np.logspace(-3, 3, 7)  # от 0.001 до 1000
```
- `l1_ratio` — параметр смешивания регуляризаторов:
$$
\rho \in [0, 1]
$$

При $\rho = 0$ используется только L2-регуляризация (Ridge),
при $\rho = 1$ — только L1 (Lasso),
промежуточные значения соответствуют ElasticNet.

В коде используется:

```python
'reg__l1_ratio': [0.1, 0.5, 0.9]
```
Это позволяет протестировать смещения в сторону Ridge, сбалансированного варианта и Lasso.

---

### GridSearchCV

```python
search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=rmsle_scorer,
    n_jobs=-1,
    verbose=1
)
```
- Используется 5-кратная кросс-валидация (`cv=5`), обеспечивающая устойчивую оценку модели на разных подвыборках.
- `scoring=rmsle_scorer` — подключается ранее определённая метрика RMSLE.
- `n_jobs=-1` — означает использование всех доступных процессоров для параллельной обработки.
- `verbose=1` — включает вывод текущего хода выполнения на экран, полезно для отладки и мониторинга.

---

### Настройки модели ElasticNet

```python
ElasticNet(max_iter=5000, tol=1e-3)
```
- `max_iter=5000` — увеличенное количество итераций метода координатного спуска. Это важно при высокоразреженных данных, где обычного количества итераций может не хватить.
- `tol=1e-3` — порог изменения функции потерь, при котором обучение прекращается. Позволяет избежать лишних итераций, если модель уже сходится.

Такие параметры обеспечивают более стабильную и точную сходимость, особенно при использовании регуляризации.

---

### Обработка результатов

После окончания обучения предсказания выполняются в логарифмической шкале:

```python
preds_log = search.predict(X_test)
preds = np.expm1(preds_log)
```
- Модель обучалась на логарифмированных значениях `y`, поэтому предсказания (`preds_log`) также находятся в логарифмической шкале.
- Функция `np.expm1(x)` выполняет обратное преобразование к `log1p(y)`, то есть вычисляет $e^x - 1$.
- Использование `expm1` более численно устойчиво, особенно при малых значениях, чем `np.exp(x) - 1`.

В результате восстанавливаются предсказанные значения в исходной шкале цен (`SalePrice`).

---

### Результат на Kaggle

После выполнения обучения и предсказания, модель ElasticNet показала следующий результат на платформе Kaggle:

> **Итоговая RMSLE: 0.12507**


## 2. CNN1D (одномерная сверточная сеть)

Сверточные нейронные сети (CNN) — класс моделей, использующих локальные фильтры для извлечения признаков из входных данных. В задаче регрессии по табличным данным CNN может использоваться как способ автоматического построения нелинейных признаков с локальной структурой. Мы применяем одномерные свёртки (Conv1D), рассматривая табличные признаки как последовательность.

---

### 1. Формализация входа

Пусть у нас есть обучающая выборка:

$$
D = \{ (x^{(i)}, y^{(i)}) \}_{i=1}^{m}
$$

где:

- $x^{(i)}$ — вектор признаков (например, 79 штук),
- $y^{(i)}$ — логарифмированная цена продажи дома,
- $m$ — количество объектов в обучающей выборке.

Вход в сверточную сеть — это $x^{(i)}$, рассматриваемый как одномерная последовательность длины $n$. Соответственно, данные имеют форму $(m, n, 1)$.

---

### 2. Свёртка (Conv1D)

Пусть:

- размер ядра (фильтра) равен \( k \),
- веса ядра — \( w = [w_1, w_2, ..., w_k] \),
- смещение — \( b \).

Тогда результат применения фильтра на позиции \( j \) равен:

$$
z_j = \sum_{i=1}^{k} w_i \cdot x_{j+i-1} + b
$$

Если применить \( F \) фильтров, результат будет матрицей:

$$
Z \in \mathbb{R}^{(n - k + 1) \times F}
$$

---

### 3. Активация

После свёртки применяется нелинейная функция, например ReLU:

$$
\text{ReLU}(z) = \max(0, z)
$$

---

### 4. Выходные признаки

Матрица \( Z \) содержит выходы всех фильтров по всем позициям:

$$
Z = [z^{(1)}, z^{(2)}, ..., z^{(F)}] \in \mathbb{R}^{(n - k + 1) \times F}
$$

---
### 5. Переход к регрессии

- Применяется Flatten:

$$
h = \text{Flatten}(Z) \in \mathbb{R}^d
$$

- Линейный выходной слой:

$$
\hat{y} = w_d^\top h + b
$$
- Или с двумя слоями и активациями:

$$
\hat{y} = \phi_2 \left( W_2^T \cdot \phi_1 \left( W_1^T \cdot h + b_1 \right) + b_2 \right)
$$

---

### 6. Функция потерь

**MSE** (если логарифмирование применено заранее):

$$
\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2
$$

**RMSLE** (если логарифмирование включено в метрику):

$$
\text{RMSLE} = \sqrt{ \frac{1}{m} \sum_{i=1}^{m} \left( \log(\hat{y}_i + 1) - \log(y_i + 1) \right)^2 }
$$

---

### 7. Почему CNN1D применим к табличным данным

- Признаки можно логически упорядочить,
- Между соседними признаками могут быть связи,
- Свёртки позволяют извлекать локальные шаблоны,
- Параметры фильтров переиспользуются, что уменьшает переобучение.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def load_data():
    train = pd.read_csv('train.csv', na_values=[''], keep_default_na=True)
    test  = pd.read_csv('test.csv',  na_values=[''], keep_default_na=True)

    for c in ['Alley','PoolQC','Fence','MiscFeature']:
        train.drop(columns=c, inplace=True)
        test .drop(columns=c, inplace=True, errors='ignore')

    for c in ['LotFrontage','MasVnrArea','GarageYrBlt']:
        m = train[c].median()
        train[c].fillna(m, inplace=True)
        test [c].fillna(m, inplace=True)

    num_cols = train.select_dtypes(include='number').columns.drop('SalePrice')
    med      = train[num_cols].median()
    train[num_cols].fillna(med, inplace=True)
    test [num_cols].fillna(med, inplace=True)

    cat_cols = train.select_dtypes(include='object').columns
    for c in cat_cols:
        train[c].fillna('None', inplace=True)
        test [c].fillna('None', inplace=True)

    ORD = {
      'ExterQual':     ["Po","Fa","TA","Gd","Ex"],
      'ExterCond':     ["Po","Fa","TA","Gd","Ex"],
      'BsmtQual':      ["None","Po","Fa","TA","Gd","Ex"],
      'BsmtCond':      ["None","Po","Fa","TA","Gd","Ex"],
      'BsmtExposure':  ["None","No","Mn","Av","Gd"],
      'BsmtFinType1':  ["None","Unf","LwQ","Rec","BLQ","ALQ","GLQ"],
      'BsmtFinType2':  ["None","Unf","LwQ","Rec","BLQ","ALQ","GLQ"],
      'HeatingQC':     ["Po","Fa","TA","Gd","Ex"],
      'KitchenQual':   ["Po","Fa","TA","Gd","Ex"],
      'FireplaceQu':   ["None","Po","Fa","TA","Gd","Ex"],
      'GarageQual':    ["None","Po","Fa","TA","Gd","Ex"],
      'GarageCond':    ["None","Po","Fa","TA","Gd","Ex"],
      'GarageFinish':  ["None","Unf","RFn","Fin"],
      'PavedDrive':    ["N","P","Y"]
    }
    for f, order in ORD.items():
        train[f] = train[f].astype('category').cat.set_categories(order).cat.codes
        if f in test:
            test[f] = test[f].astype('category').cat.set_categories(order).cat.codes

    rest = train.select_dtypes(include='object').columns
    train = pd.get_dummies(train, columns=rest, drop_first=True)
    test  = pd.get_dummies(test,  columns=rest, drop_first=True)

    train, test = train.align(test, join='left', axis=1, fill_value=0)
    test.drop(columns=['SalePrice'], inplace=True, errors='ignore')

    y = np.log1p(train.pop('SalePrice'))

    nums = train.select_dtypes(include='number').columns.drop('Id')
    pt = PowerTransformer(method='yeo-johnson', standardize=False)
    train[nums] = pt.fit_transform(train[nums])
    test [nums] = pt.transform(test[nums])
    scaler = StandardScaler()
    train[nums] = scaler.fit_transform(train[nums])
    test [nums] = scaler.transform(test[nums])

    X      = train.drop(columns=['Id']).values.astype('float32')[..., None]
    X_test = test .drop(columns=['Id']).values.astype('float32')[..., None]

    return X, y.values.astype('float32'), X_test

X, y, X_test = load_data()
n_feat = X.shape[1]

def build_model():
    inp = layers.Input((n_feat, 1))

    x = layers.Conv1D(64, 5, padding='same')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv1D(128, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.GlobalAveragePooling1D()(x)

    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(64)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)

    out = layers.Dense(1)(x)

    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

kf  = KFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros_like(y)

for fold, (tr, va) in enumerate(kf.split(X)):
    X_tr, X_va = X[tr], X[va]
    y_tr, y_va = y[tr], y[va]

    tf.random.set_seed(100 + fold)
    m = build_model()

    es = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

    m.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=100,
        batch_size=64,
        callbacks=[es, rl],
        verbose=0
    )

    oof[va] = m.predict(X_va).flatten()

print("OOF RMSLE:", np.sqrt(mean_squared_error(y, oof)))

tf.random.set_seed(999)
final = build_model()
es = callbacks.EarlyStopping(monitor='loss', patience=7, restore_best_weights=True)
rl = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1)

final.fit(X, y, epochs=100, batch_size=64, callbacks=[es, rl], verbose=0)

preds_log = final.predict(X_test).flatten()
preds     = np.expm1(preds_log)

sub = pd.read_csv('sample_submission.csv')
sub['SalePrice'] = pd.to_numeric(preds, errors='coerce')
median_price    = sub['SalePrice'].median()
sub['SalePrice'].fillna(median_price, inplace=True)
sub['SalePrice'] = sub['SalePrice'].astype(float)
sub.to_csv('submission_cnn1d_best.csv', index=False)
print("Saved submission_cnn1d_best.csv")

## Подробности реализации модели CNN1D на основе кода
В данной реализации используется одномерная сверточная нейронная сеть (CNN1D), применённая к табличным данным. Модель реализована с использованием Keras (TensorFlow backend), с полной предобработкой и отладочной кросс-валидацией.
___
### Архитектура модели
Модель строится в функции build_model() и включает следующие блоки:
```python
inp = layers.Input((n_feat, 1))
```
- **Вход**: одномерный тензор размера `(n_feat, 1)` — каждый признак интерпретируется как временная точка.

**Свёрточный блок 1:**
```python
x = layers.Conv1D(64, 5, padding='same')(inp)
```
- **64 фильтра**, каждый шириной 5, шаг по умолчанию — 1;
- padding='same' сохраняет длину выхода.
```python
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(0.3)(x)
```
- **BatchNorm** стабилизирует обучение;
- **ReLU** — нелинейная активация;
- **MaxPooling1D(2)** уменьшает размерность вдвое;
- **Dropout(0.3)** — регуляризация, отключает 30% нейронов.

**Свёрточный блок 2:**
```python
x = layers.Conv1D(128, 3, padding='same')(x)
```
- **128 фильтров**, ширина фильтра — 3.
```python
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(0.3)(x)
```
— те же операции, но с другим числом фильтров и уменьшенной шириной.

**Переход к dense-части:**
```python
x = layers.GlobalAveragePooling1D()(x)
```
- **Глобальный пулинг** — берёт среднее по всей временной оси для каждого фильтра, сильно уменьшает размерность и повышает устойчивость.
___

### Полносвязные слои
```python
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.4)(x)
```
- Полносвязный слой на 256 нейронов;
- Dropout (0.4) — сильная регуляризация (40%).
```python
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.2)(x)
```
- Ещё один Dense (64 нейрона) + регуляризация.
```python
out = layers.Dense(1)(x)
```
- Один выходной нейрон — регрессионный выход (логарифм цены дома).
___
### Компиляция модели
```python
model.compile(optimizer='adam', loss='mse')
```
- **Adam** — адаптивный оптимизатор (изменяет шаг обучения для каждого веса);
- **MSE** — среднеквадратичная ошибка на логарифмах цен.
___
### Стратегия обучения
#### Используется 5-кратная кросс-валидация (KFold):
```python
kf = KFold(n_splits=5, shuffle=True, random_state=42)
```
- shuffle=True + seed — для стабильности;
- обучаем модель на 4 фолдах, валидируем на 5-м.

#### EarlyStopping
```python
callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
```
- Останавливает обучение, если val_loss не улучшается 7 эпох подряд;
- restore_best_weights=True — возвращает модель к лучшему состоянию.

#### ReduceLROnPlateau
```python
callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
```
- Если val_loss не улучшается 5 эпох, learning rate уменьшается вдвое;
- Способ адаптивной регулировки скорости обучения.
___
### Обучение
```python
m.fit(..., epochs=100, batch_size=64, ...)
```
- Модель обучается максимум 100 эпох;
- Размер батча: 64;
- Реально обучение завершается раньше благодаря EarlyStopping.
___
### Финальное предсказание
После валидации на 5 фолдах, модель переобучается на всех данных (X, y), предсказывает X_test.
```python
preds_log = final.predict(X_test).flatten()
preds = np.expm1(preds_log)
```
- Предсказания в лог-шкале → восстанавливаем реальные цены через expm1.
___
### Результат на Kaggle

После выполнения обучения и предсказания, модель ElasticNet показала следующий результат на платформе Kaggle:

> **Итоговая RMSLE: 0.56126**

## MLP (полносвязная нейронная сеть)
MLP (Multi-Layer Perceptron) — классическая нейронная сеть, в которой каждый нейрон на слое связан со всеми нейронами предыдущего слоя. В контексте регрессии по табличным данным MLP обучается преобразовывать вектор признаков в непрерывное значение — прогноз логарифма цены.
___
### 1. Формализация входа
Пусть обучающая выборка имеет вид:

$$
D = { (x^{(i)}, y^{(i)}) }_{i=1}^{m}
$$

где:\
	•	$x^{(i)} \in \mathbb{R}^n$ — вектор признаков,\
	•	$y^{(i)} \in \mathbb{R}$ — логарифм цены (целевое значение),\
	•	$m$ — число объектов.

Модель принимает входной вектор x и последовательно преобразует его с помощью нескольких полносвязных слоёв и активаций.
___
### 2. Архитектура сети
Общая схема MLP выглядит так:

$$
\begin{aligned}
h^{(1)} &= \phi_1(W^{(1)} x + b^{(1)}) \\
h^{(2)} &= \phi_2(W^{(2)} h^{(1)} + b^{(2)}) \\
&\vdots \\
h^{(L-1)} &= \phi_{L-1}(W^{(L-1)} h^{(L-2)} + b^{(L-1)}) \\
\hat{y} &= W^{(L)} h^{(L-1)} + b^{(L)}
\end{aligned}
$$

где:\
	•	$W^{(l)}$ — матрица весов слоя $l$,\
	•	$b^{(l)}$ — вектор смещений,\
	•	$\phi_l$ — функция активации (обычно ReLU),\
	•	$L$ — количество слоёв (включая выходной),\
	•	$h^{(l)}$ — активации на слое $l$,\
	•	$\hat{y}$ — предсказание (лог-цена).
___
### 3. Активации
На скрытых слоях используются нелинейные функции:

$$
\phi(x) = \max(0, x) \quad \text{(ReLU)}
$$

Эта функция помогает сети моделировать нелинейные зависимости между признаками.
___
### 4. Регуляризация и нормализация
Чтобы повысить устойчивость модели, применяются:
- Dropout — случайное зануление нейронов:
Например, Dropout(0.4) означает, что 40% выходов слоя обнуляются случайно при обучении.
- Batch Normalization — нормализация выходов слоя:
$$
\text{BN}(x) = \frac{x - \mu}{\sqrt{\sigma^2 + \varepsilon}}
$$
где $\mu$ и $\sigma^2$ — среднее и дисперсия по батчу, $\varepsilon$ — маленькая константа для численной устойчивости.

Это ускоряет обучение и делает модель менее чувствительной к выбору параметров.
___
### 5. Выходной слой
Выходной слой состоит из одного нейрона без активации:

$$
\hat{y} = w^\top h^{(L-1)} + b
$$

где $h^{(L-1)}$ — последняя скрытая репрезентация, $w$ и $b$ — параметры.
___
### 6. Функция потерь
Для обучения используется среднеквадратичная ошибка (MSE):

$$
\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2
$$

Если логарифмирование целевой переменной уже применено (как в нашем случае), RMSLE не используется, и MSE работает как прямая метрика.
___
### 7. Почему MLP применим к табличным данным
- Способен моделировать как линейные, так и сложные нелинейные зависимости,
- Не требует упорядоченности признаков,
- Легко масштабируется на большие таблицы,
- Работает «в лоб» — просто принимает вектор признаков и выдаёт число.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
import warnings

warnings.filterwarnings('ignore')

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

train = pd.read_csv('train.csv', na_values=[''], keep_default_na=True)
test  = pd.read_csv('test.csv',  na_values=[''], keep_default_na=True)
test_ID = test['Id']
y = np.log1p(train['SalePrice'])
train.drop(columns=['SalePrice'], inplace=True)
data = pd.concat([train, test], ignore_index=True)

num_feats = data.select_dtypes(['int64','float64']).columns.drop('Id')
skews = data[num_feats].apply(lambda x: skew(x.dropna()))
skewed = skews[abs(skews) > 0.75].index
data[skewed] = np.log1p(data[skewed])

cat_feats = data.select_dtypes(['object']).columns.tolist()

num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale',  StandardScaler())
])
cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe',    OneHotEncoder(handle_unknown='ignore'))
])

preproc = ColumnTransformer([
    ('num', num_pipe, num_feats),
    ('cat', cat_pipe, cat_feats)
])

mlp = Pipeline([
    ('prep', preproc),
    ('model', MLPRegressor(hidden_layer_sizes=(512,256,128),
                           activation='relu',
                           solver='adam',
                           alpha=1e-4,
                           learning_rate_init=1e-3,
                           max_iter=200,
                           random_state=42))
])

param_grid = {
    'model__alpha': [1e-4, 1e-3, 1e-2],
    'model__learning_rate_init': [1e-3, 1e-2]
}
search = GridSearchCV(mlp, param_grid, cv=5, scoring=rmsle_scorer, n_jobs=-1, verbose=1)
X_train = data.iloc[:len(y)].drop(columns=['Id'])
search.fit(X_train, y)
print("MLP best params:", search.best_params_)

X_test = data.iloc[len(y):].drop(columns=['Id'])
preds_log = search.predict(X_test)
preds = np.expm1(preds_log)

sub = pd.read_csv('sample_submission.csv')
sub['SalePrice'] = preds
sub.to_csv('submission_mlp.csv', index=False)
print("Saved submission_mlp.csv")

## Подробности реализации модели MLP на основе кода
В данной реализации используется классическая полносвязная нейронная сеть (MLP) из sklearn.neural_network.MLPRegressor, встроенная в Pipeline с полной предобработкой данных и автоматическим подбором гиперпараметров.
___
### Предобработка данных
Масштабирование и кодирование признаков выполняется с помощью ColumnTransformer:
```python
preproc = ColumnTransformer([
    ('num', num_pipe, num_feats),
    ('cat', cat_pipe, cat_feats)
])
```
- Числовые признаки:
    - Заполняются медианой (SimpleImputer(strategy='median'))
    - Масштабируются (StandardScaler)
- Категориальные признаки:
    - Заполняются наиболее частым значением (SimpleImputer(strategy='most_frequent'))
    - Кодируются с помощью OneHotEncoder (игнорируя неизвестные значения)

Перед этим:
- Числовые признаки со скошенным распределением логарифмируются с помощью np.log1p(), если $|\text{skew}| > 0.75$.
___
### Архитектура модели
```python
MLPRegressor(
    hidden_layer_sizes=(512, 256, 128),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    learning_rate_init=1e-3,
    max_iter=200,
    random_state=42
)
```
- **Слои:** три скрытых слоя с 512, 256 и 128 нейронами.
- **Активация:** ReLU — стандартная нелинейность $\phi(x) = \max(0, x)$.
- **Оптимизатор:** adam — адаптивный стохастический градиент.
- **Регуляризация:** alpha=1e-4 — L2-регуляризация (аналог weight decay).
- **Начальный learning rate:** 1e-3 — базовая скорость обучения.
- **Максимум итераций:** 200 эпох (может не сойтись, если данные сложные).
- **random_state=42** — для воспроизводимости результата.
___
### Подбор гиперпараметров

Подбор осуществляется с помощью GridSearchCV по сетке:
```python
param_grid = {
    'model__alpha': [1e-4, 1e-3, 1e-2],
    'model__learning_rate_init': [1e-3, 1e-2]
}
```
- Подбираются два параметра:
    - alpha — коэффициент регуляризации;
    - learning_rate_init — начальная скорость обучения.
- Используется 5-кратная кросс-валидация (cv=5);
- Метрика: rmsle_scorer — кастомная RMSLE-функция (при логарифмированной y):
```python
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
```
___
### Сабмит и восстановление цен
После получения логарифмированных предсказаний:
```python
preds_log = search.predict(X_test)
preds = np.expm1(preds_log)
```
- Используется np.expm1 — точное восстановление из log1p;
- Создаётся файл submission_mlp.csv в требуемом формате.

___
### Результат на Kaggle
Несмотря на простоту архитектуры и ограниченные возможности настройки через MLPRegressor, модель достигла на Kaggle результата:

> **Итоговая RMSLE: 0.15466**

Для модели без ручной настройки глубокой архитектуры и встроенного early stopping — это вполне достойный результат. Улучшение возможно при переходе к фреймворкам с полной поддержкой обучения нейросетей (Keras, PyTorch).

## Вывод
После обучения и тестирования трёх различных архитектур были получены следующие значения метрики **RMSLE** на Kaggle:

| Модель                  | RMSLE       |
|------------------------|-------------|
| **ElasticNet**         | **0.12507** |
| **MLP (полносвязная)** | 0.15466     |
| **CNN1D (свёрточная)** | 0.56126     |

---
- **ElasticNet** показал **наилучший результат**, несмотря на простую линейную природу. Это объясняется тем, что:
  - данные хорошо масштабированы и подготовлены;
  - логарифмирование целевой переменной снизило влияние выбросов;
  - регуляризация L1/L2 позволила избежать переобучения;
  - модель хорошо справляется с табличными данными, где признаки независимы.

- **MLP** (нейронная сеть с 3 слоями) занял **второе место**. Он смог уловить более сложные зависимости, чем линейная модель, но:
  - требует точной настройки гиперпараметров;
  - чувствителен к переобучению;
  - хуже интерпретируем.

- **CNN1D** показал себя **наихудше** на табличных данных:
  - несмотря на глубокую архитектуру, модель не смогла эффективно использовать локальные зависимости;
  - признаки в таблицах не обладают пространственной структурой, необходимой для свёрток;
  - архитектура CNN требует существенно другой подход к представлению данных (например, графовые или временные зависимости).

---
Это задание стало моим первым практическим знакомством с нейросетевыми архитектурами. Процесс оказался местами сложным и не всегда понятным: особенно в части настройки параметров и понимания математики, лежащей в основе моделей. Для повышения качества в будущем мне необходимо глубже разобраться:
- в математической сути нейронных сетей;
- в принципах регуляризации и нормализации;
- в подходах к поиску оптимальных архитектур и гиперпараметров.

Тем не менее, этот проект дал мне базовое понимание различий между классическими и нейросетевыми методами, а также важности предобработки при работе с табличными данными.
